In [4]:
#__________________________________________chargement data.xls__________________________________________________________________
import pandas as pd
import pyodbc

# Fonction pour charger un fichier Excel
def load_excel(file_path):
    # Lire le fichier Excel avec pandas
    df = pd.read_excel(file_path, sheet_name=None)  # 'sheet_name=None' charge toutes les feuilles
    return df

# Fonction pour convertir les colonnes datetime en chaîne ou datetime si nécessaire
def convert_datetime_columns(df):
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            # Convertir les colonnes datetime en string (ou vous pouvez utiliser 'DATE' ou 'DATETIME' selon le besoin)
            df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S')  # format datetime en string
    return df

# Fonction pour nettoyer les colonnes "Unnamed" et les renommer
def clean_unnamed_columns(df):
    # Supprimer les colonnes qui ont "Unnamed" dans leur nom
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# Fonction pour charger les données dans SQL Server
def load_to_sql_server(df, table_name, server, database, username, password):
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Supprimer la table si elle existe
        cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};")
        conn.commit()
        
        # Créer la table avec des types adaptés
        create_table_query = f"""
        CREATE TABLE {table_name} (
        {', '.join([f'[{col}] NVARCHAR(MAX)' if df[col].dtype == object else f'[{col}] FLOAT' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        
        # Insérer les données dans la table
        for index, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            try:
                values = [None if pd.isna(value) else value for value in row.values]
                cursor.execute(insert_query, values)
            except Exception as e:
                print(f"Erreur lors de l'insertion de la ligne {index} : {e}")
        
        conn.commit()
        print(f"Données insérées avec succès dans la table {table_name}.")
    
    except Exception as e:
        print(f"Erreur globale : {e}")
    finally:
        cursor.close()
        conn.close()

# Charger le fichier Excel
file_path = r"C:\Users\user\Downloads\Projet 2024\Data.xlsx"
df = load_excel(file_path)

# Afficher les noms des feuilles disponibles pour identifier la bonne feuille
print("Feuilles disponibles dans le fichier Excel :", list(df.keys()))

# Vérifier si une feuille spécifique existe et charger les données
sheet_name = 'Feuil2'  # Remplacer par le nom de la feuille que vous voulez utiliser
if sheet_name in df:
    df_sheet = df[sheet_name]
    print(f"Chargement des données de la feuille '{sheet_name}'.")
    print(f"Exemple de données de la feuille '{sheet_name}':\n", df_sheet.head())  # Affiche un échantillon des données
else:
    print(f"La feuille '{sheet_name}' n'existe pas dans le fichier Excel. Voici les feuilles disponibles : {list(df.keys())}")
    exit()

# Nettoyer les colonnes "Unnamed"
df_sheet = clean_unnamed_columns(df_sheet)

# Convertir les colonnes datetime
df_sheet = convert_datetime_columns(df_sheet)

# Vérifier à nouveau les types de colonnes
print(f"Types de colonnes de la feuille '{sheet_name}':\n", df_sheet.dtypes)

# Charger les données dans SQL Server
load_to_sql_server(
    df=df_sheet,
    table_name=f'{sheet_name}',  # Nom de la table dans SQL Server
    server='DESKTOP-U56S90R',
    database='KomaxData',
    username='sa',
    password='yosr'
)


Feuilles disponibles dans le fichier Excel : ['Equipement', 'Feuil2']
Chargement des données de la feuille 'Feuil2'.
Exemple de données de la feuille 'Feuil2':
        ARTI_CODE  ARTI_TCY_MOYEN ARTI_CALC_DELAI_MODE ARTI_CALC_DELAI  \
0            NaN             NaN                  NaN             NaN   
1        4080052             NaN                  NaN               N   
2      612085001             NaN                  NaN               N   
3       GC6X4060             NaN                  NaN               N   
4  KCP-612188000             NaN                  NaN               N   

                           ARTI_DESIGNATION ARTI_SEVE_CODE ARTI_INDICE  \
0                                       NaN            NaN         NaN   
1             HARNESS EASY FAST 8 CIL. OBII            NaN           1   
2           CABL.CAVET.INTER CONNECT.SICMA2            NaN           3   
3  GAINE EN PVC DIAM 6MM NOIRE 105° L4060MM            NaN         NaN   
4     KCP-612188000 CABLAGE EL

In [5]:
#_________________________________chargement Job______________________________________________________
import pandas as pd
import os
import zipfile
import tempfile
import pyodbc

# Fonction pour analyser et structurer les données pour IDC
def parse_idc_data(text):
    data = []
    lines = text.strip().splitlines()
    
    for line in lines:
        columns = line.split('\t')
        if len(columns) >= 6:
            # Vérifier et séparer les valeurs contenant des virgules dans des colonnes séparées
            identifier_values = columns[4].split(',') if ',' in columns[4] else [columns[4]]
            status_values = columns[5].split(',') if ',' in columns[5] else [columns[5]]
            
            # Ajouter des colonnes supplémentaires si nécessaire
            row = {
                'Date': columns[0],
                'Time': columns[1],
                'Section': columns[2],
                'Action': columns[3],
                'Identifier1': identifier_values[0],  # Première valeur de l'identifiant
                'Identifier2': identifier_values[1] if len(identifier_values) > 1 else None,  # Seconde valeur de l'identifiant (si elle existe)
                'Status1': status_values[0],  # Première valeur du statut
                'Status2': status_values[1] if len(status_values) > 1 else None,  # Seconde valeur du statut (si elle existe)
            }
            data.append(row)
    
    return pd.DataFrame(data)

# Fonction pour analyser et structurer les données pour LDC
def parse_ldc_data(text):
    data = []
    lines = text.strip().splitlines()

    for line in lines:
        columns = line.split('\t')  # Séparation par tabulation
        if len(columns) >= 6:
            # Vérifier et séparer les valeurs contenant des virgules dans des colonnes séparées
            identifier_values = columns[4].split(',') if ',' in columns[4] else [columns[4]]
            status_values = columns[5].split(',') if ',' in columns[5] else [columns[5]]
            
            # Ajouter des colonnes supplémentaires si nécessaire
            row = {
                'Date': columns[0],
                'Time': columns[1],
                'Section': columns[2],
                'Action': columns[3],
                'Identifier1': identifier_values[0],  # Première valeur de l'identifiant
                'Identifier2': identifier_values[1] if len(identifier_values) > 1 else None,  # Seconde valeur de l'identifiant (si elle existe)
                'Status1': status_values[0],  # Première valeur du statut
                'Status2': status_values[1] if len(status_values) > 1 else None,  # Seconde valeur du statut (si elle existe)
            }
            data.append(row)

    return pd.DataFrame(data)

# Fonction pour traiter les fichiers ZIP et détecter le type de fichier (IDC ou LDC)
def process_zip(zip_path):
    all_data = []
    inserted_files = []  # Liste pour stocker les fichiers insérés
    columns_set = set()  # Set pour collecter toutes les colonnes rencontrées
    
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        for root, dirs, files in os.walk(temp_dir):
            for file in files:
                file_path = os.path.join(root, file)
                if file.lower().endswith(('.idc', '.ldc')):  # Vérification insensible à la casse
                    print(f"Fichier trouvé : {file_path}")  # Affiche le chemin du fichier trouvé
                    try:
                        with open(file_path, 'r') as f:
                            text = f.read()

                        if 'idc' in file.lower():  # Si le fichier est IDC
                            df = parse_idc_data(text)
                        else:  # Si le fichier est LDC
                            df = parse_ldc_data(text)

                        # Ajouter la colonne FilePath pour suivre le fichier
                        df['FilePath'] = file_path
                        all_data.append(df)
                        
                        # Ajouter à la liste des fichiers insérés
                        inserted_files.append(file_path)

                        # Mettre à jour le set des colonnes rencontrées
                        columns_set.update(df.columns)

                    except Exception as e:
                        print(f"Erreur lors du traitement du fichier {file_path}: {e}")
    
    # Retourner les données et les fichiers insérés
    if all_data:
        # Aligner les colonnes
        for i, df in enumerate(all_data):
            missing_columns = columns_set - set(df.columns)  # Colonnes manquantes
            for col in missing_columns:
                df[col] = None  # Ajouter des colonnes manquantes avec des valeurs None
            all_data[i] = df[list(columns_set)]  # Réorganiser les colonnes pour être les mêmes partout
        
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df, inserted_files
    else:
        return pd.DataFrame(), inserted_files

# Fonction pour nettoyer et préparer le DataFrame avant insertion dans SQL Server
def clean_dataframe(df):
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].fillna("")
        elif pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(None)
    return df

# Fonction pour charger les données dans SQL Server
def load_to_sql_server(df, table_name, server, database, username, password, inserted_files):
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()

        # Supprimer la table si elle existe
        drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};"
        cursor.execute(drop_table_query)
        conn.commit()

        # Créer la table avec des colonnes dynamiques
        create_table_query = f"""
        CREATE TABLE {table_name} (
        {', '.join([f'[{col}] NVARCHAR(MAX)' if df[col].dtype == object else f'[{col}] FLOAT' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)
        conn.commit()

        # Insérer les données ligne par ligne
        for index, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            cursor.execute(insert_query, *row.values.tolist())
        
        conn.commit()
        print(f"Données insérées avec succès dans la table {table_name}.")
        
        # Afficher les fichiers insérés
        print("Fichiers insérés dans la table :")
        for file in inserted_files:
            print(f"Fichier inséré : {file}")
    
    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")
    
    finally:
        cursor.close()
        conn.close()


# Exemple de chemin vers le fichier ZIP
zip_path = r'C:/Users/user/Downloads/Projet 2024.zip'

# Traiter l'archive ZIP
df_combined, inserted_files = process_zip(zip_path)

# Charger les données dans SQL Server
if not df_combined.empty:
    # Nettoyer le DataFrame
    df_combined = clean_dataframe(df_combined)
    
    load_to_sql_server(
        df=df_combined,
        table_name='Job',  # Nouveau nom de la table
        server='DESKTOP-U56S90R',
        database='KomaxData',
        username='sa',
        password='yosr',
        inserted_files=inserted_files  # Passer les fichiers insérés à la fonction
    )


Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\01\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\04\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\05\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\06\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\07\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\08\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\11\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\11\Terminal.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\12\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\13\Job.LDC
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9q0o_kd\Data_komax\433\11\14\Jo

In [6]:
#___________________________________________________chargement Equipement_____________________________________________________
import pandas as pd
import pyodbc

# Fonction pour charger un fichier Excel
def load_excel(file_path):
    # Lire le fichier Excel avec pandas
    df = pd.read_excel(file_path, sheet_name=None)  # 'sheet_name=None' charge toutes les feuilles
    return df

# Fonction pour convertir les colonnes datetime en chaîne ou datetime si nécessaire
def convert_datetime_columns(df):
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            # Convertir les colonnes datetime en string (ou vous pouvez utiliser 'DATE' ou 'DATETIME' selon le besoin)
            df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S')  # format datetime en string
    return df

# Fonction pour charger les données dans SQL Server
def load_to_sql_server(df, table_name, server, database, username, password):
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Supprimer la table si elle existe
        cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};")
        conn.commit()
        
        # Créer la table avec des types adaptés
        create_table_query = f"""
        CREATE TABLE {table_name} (
        {', '.join([f'[{col}] NVARCHAR(MAX)' if df[col].dtype == object else f'[{col}] FLOAT' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        
        # Insérer les données dans la table
        for index, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            try:
                values = [None if pd.isna(value) else value for value in row.values]
                cursor.execute(insert_query, values)
            except Exception as e:
                print(f"Erreur lors de l'insertion de la ligne {index} : {e}")
        
        conn.commit()
        print(f"Données insérées avec succès dans la table {table_name}.")
    
    except Exception as e:
        print(f"Erreur globale : {e}")
    finally:
        cursor.close()
        conn.close()

# Charger le fichier Excel
file_path = r"C:\Users\user\Downloads\Projet 2024\Data.xlsx"
df = load_excel(file_path)

# Afficher les noms des feuilles disponibles pour identifier la bonne feuille
print("Feuilles disponibles dans le fichier Excel :", list(df.keys()))

# Vérifier si une feuille spécifique existe
sheet_name = 'Equipement'  # Remplacez par le nom de la feuille que vous voulez utiliser
if sheet_name in df:
    df_sheet = df[sheet_name]
    print(f"Chargement des données de la feuille '{sheet_name}'.")
else:
    print(f"La feuille '{sheet_name}' n'existe pas dans le fichier Excel. Voici les feuilles disponibles : {list(df.keys())}")
    exit()

# Convertir les colonnes datetime
df_sheet = convert_datetime_columns(df_sheet)

# Charger les données dans SQL Server
load_to_sql_server(
    df=df_sheet,
    table_name='Equipement_Table',  # Nom de la table dans SQL Server
    server='DESKTOP-U56S90R',
    database='KomaxData',
    username='sa',
    password='yosr'
)

Feuilles disponibles dans le fichier Excel : ['Equipement', 'Feuil2']
Chargement des données de la feuille 'Equipement'.
Données insérées avec succès dans la table Equipement_Table.


In [1]:
#_____________________________________________table BESOIN_OF_H_DATA_TABLE_____________________________________________________________________
import pandas as pd
import numpy as np
import pyodbc
import re

# Fonction pour détecter et corriger les erreurs dans le fichier CSV
def load_and_clean_csv(file_path):
    # Charger le fichier en tant que texte pour détecter les lignes problématiques
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Remplacer les séquences de virgules multiples (',,,') par une seule virgule
    cleaned_lines = []
    for line in lines:
        # Remplacer les virgules consécutives par une seule virgule
        line = re.sub(r',+', ',', line)
        cleaned_lines.append(line)
    
    # Sauvegarder le fichier nettoyé temporairement
    cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')
    with open(cleaned_file_path, 'w', encoding='utf-8') as file:
        file.writelines(cleaned_lines)
    
    # Charger le fichier nettoyé dans un DataFrame
    df = pd.read_csv(cleaned_file_path, low_memory=False)
    return df

# Fonction pour nettoyer le DataFrame
def clean_dataframe(df):
    for col in df.columns:
        if df[col].dtype == object:
            # Remplacer les NaN dans les colonnes texte par une chaîne vide
            df[col] = df[col].fillna("").astype(str)
        elif pd.api.types.is_numeric_dtype(df[col]):
            # Remplacer les NaN et valeurs infinies dans les colonnes numériques
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertit les erreurs en NaN
            df[col] = df[col].fillna(np.nan).replace([np.inf, -np.inf], np.nan)
    return df

# Fonction pour charger les données dans SQL Server
def load_to_sql_server(df, table_name, server, database, username, password):
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Supprimer la table si elle existe
        drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};"
        cursor.execute(drop_table_query)
        conn.commit()
        
        # Création de la table
        create_table_query = f"""
        CREATE TABLE {table_name} (
        {', '.join([f'[{col}] FLOAT' if pd.api.types.is_numeric_dtype(df[col]) else f'[{col}] NVARCHAR(MAX)' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        
        # Insérer les données ligne par ligne
        for index, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            try:
                cursor.execute(insert_query, *[None if pd.isna(value) else value for value in row.values])
            except Exception as e:
                print(f"Erreur lors de l'insertion de la ligne {index}: {e}")
        
        conn.commit()
        print(f"Données insérées avec succès dans la table {table_name}.")
    
    except Exception as e:
        print(f"Erreur globale : {e}")
    finally:
        cursor.close()
        conn.close()

# Charger le fichier CSV et corriger les erreurs
csv_file_path = r"C:\Users\user\Desktop\developer\sqldeveloper\sqldeveloper\bin\BESOIN_OF_H_DATA_TABLE.csv"
df = load_and_clean_csv(csv_file_path)

# Nettoyer le DataFrame
df_cleaned = clean_dataframe(df)

# Charger les données dans SQL Server
load_to_sql_server(
    df=df_cleaned,
    table_name='BESOIN_OF_H_DATA_TABLEeeeeee',  # Nom de la table
    server='DESKTOP-U56S90R',
    database='Data_Komax',
    username='sa',
    password='yosr'
)


Données insérées avec succès dans la table BESOIN_OF_H_DATA_TABLE.


In [2]:
#___________________________________________________BESOIN_OF_GROF_H_DATA_TABLE_________________________________________________________________________
import pandas as pd
import numpy as np
import pyodbc

# Fonction pour nettoyer le DataFrame
def clean_dataframe(df):
    for col in df.columns:
        if df[col].dtype == object:
            # Remplacer les NaN dans les colonnes texte par une chaîne vide
            df[col] = df[col].fillna("").astype(str)
        elif pd.api.types.is_numeric_dtype(df[col]):
            # Remplacer les NaN et valeurs infinies dans les colonnes numériques
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertit les erreurs en NaN
            df[col] = df[col].fillna(np.nan).replace([np.inf, -np.inf], np.nan)
    return df

# Fonction pour charger les données dans SQL Server
def load_to_sql_server(df, table_name, server, database, username, password):
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Supprimer la table si elle existe
        drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};"
        cursor.execute(drop_table_query)
        conn.commit()
        
        # Création de la table
        create_table_query = f"""
        CREATE TABLE {table_name} (
        {', '.join([f'[{col}] FLOAT' if pd.api.types.is_numeric_dtype(df[col]) else f'[{col}] NVARCHAR(MAX)' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        
        # Insérer les données ligne par ligne
        for index, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            try:
                cursor.execute(insert_query, *[None if pd.isna(value) else value for value in row.values])
            except Exception as e:
                print(f"Erreur lors de l'insertion de la ligne {index}: {e}")
        
        conn.commit()
    
    except Exception as e:
        print(f"Erreur globale : {e}")
    finally:
        cursor.close()
        conn.close()

# Charger le fichier TSV (en utilisant un délimiteur tabulation)
tsv_file_path = r"C:\Users\user\Desktop\developer\sqldeveloper\sqldeveloper\bin\BESOIN_OF_GROF_H_DATA_TABLE.tsv"
df = pd.read_csv(tsv_file_path, sep='\t', low_memory=False)  # Spécifier le séparateur tabulation

# Nettoyer le DataFrame
df_cleaned = clean_dataframe(df)

# Charger les données dans SQL Server
load_to_sql_server(
    df=df_cleaned,
    table_name='BESOIN_OF_GROF_H_DATA_TABLE',
    server='DESKTOP-U56S90R',
    database='KomaxData',
    username='sa',
    password='yosr'
)


In [1]:
#____________________________________________________chargement KomaxData_______________________________________________________________________________
import pandas as pd
import os
import re
import zipfile
import tempfile
import pyodbc  # Pour la connexion à SQL Server

# Fonction pour analyser et structurer les données
def parse_data(text):
    data = []
    sections = re.split(r'\[([^\]]+)\]', text.strip())  # Diviser par sections
    last_row = None  # Pour conserver la ligne précédente

    for i in range(1, len(sections), 2):
        section_name = sections[i].strip()  # Le nom de la section
        content = sections[i + 1].strip().splitlines()  # Contenu de la section

        if section_name.lower() == "counter" and last_row is not None:
            # Ajouter les données de la section "Counter" à la dernière ligne
            for line in content:
                if '=' in line:
                    key, value = line.split('=', 1)
                    values = [v.strip() for v in value.split(',')]

                    # Ajouter chaque valeur comme colonne unique
                    if len(values) > 1:
                        for idx, v in enumerate(values, start=1):
                            col_name = f"{key.strip()}_{idx}"  # Nom de colonne unique
                            last_row[col_name] = v
                    else:
                        last_row[key.strip()] = value.strip()
        else:
            # Créer une nouvelle ligne pour les autres sections
            row = {"Section": section_name}
            for line in content:
                if '=' in line:
                    key, value = line.split('=', 1)
                    values = [v.strip() for v in value.split(',')]

                    # Ajouter chaque valeur comme colonne unique
                    if len(values) > 1:
                        for idx, v in enumerate(values, start=1):
                            col_name = f"{key.strip()}_{idx}"
                            row[col_name] = v
                    else:
                        row[key.strip()] = value.strip()

            data.append(row)
            last_row = row  # Mettre à jour la dernière ligne

    return pd.DataFrame(data)

# Fonction pour nettoyer le DataFrame
def clean_dataframe(df):
    """
    Nettoyer et préparer le DataFrame avant insertion dans SQL Server.
    - Convertir les colonnes en types compatibles.
    - Remplacer uniquement les valeurs NaN par des valeurs appropriées.
    """
    for col in df.columns:
        if df[col].dtype == object:
            # Remplacer les NaN dans les colonnes texte par une chaîne vide
            df[col] = df[col].fillna("")
        elif pd.api.types.is_numeric_dtype(df[col]):
            # Remplacer les NaN dans les colonnes numériques par None (géré par SQL comme NULL)
            df[col] = df[col].fillna(None)

    return df

# Fonction pour traiter les fichiers dans un fichier ZIP
def process_zip(zip_path):
    all_data = []

    # Créer un répertoire temporaire pour extraire les fichiers ZIP
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Explorer les fichiers extraits
        for root, dirs, files in os.walk(temp_dir):
            print(f"Exploration : {root}")
            for file in files:
                if file.endswith(('.sdc', '.SDC')):  # Traiter les fichiers avec .sdc ou .SDC
                    file_path = os.path.join(root, file)
                    print(f"Fichier trouvé : {file_path}")

                    # Lire le contenu du fichier
                    try:
                        with open(file_path, 'r') as f:
                            text = f.read()

                        # Appliquer la fonction de parsing
                        df = parse_data(text)
                        df['FilePath'] = file_path  # Ajouter une colonne avec le chemin du fichier
                        all_data.append(df)

                    except Exception as e:
                        print(f"Erreur lors du traitement du fichier {file_path}: {e}")

    # Combiner toutes les données en un seul DataFrame
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print("\n--- Données combinées ---")
        print(combined_df.head())  # Afficher les premières lignes du DataFrame

        return combined_df
    else:
        print("Aucune donnée trouvée pour les fichiers .sdc")
        return pd.DataFrame()

# Fonction pour charger les données dans SQL Server
def load_to_sql_server(df, table_name, server, database, username, password):
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    try:
        # Connexion à la base de données
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()

        # Supprimer la table si elle existe
        drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};"
        cursor.execute(drop_table_query)
        conn.commit()
        print(f"Table {table_name} supprimée si elle existait.")

        # Création de la table
        create_table_query = f"""
        CREATE TABLE {table_name} (
        {', '.join([f'[{col}] FLOAT' if pd.api.types.is_numeric_dtype(df[col]) else f'[{col}] NVARCHAR(MAX)' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table {table_name} créée avec succès.")

        # Insérer les données ligne par ligne
        for index, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            cursor.execute(insert_query, *row.values.tolist())

        conn.commit()
        print(f"Données insérées avec succès dans la table {table_name}.")

    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")

    finally:
        cursor.close()
        conn.close()

# Chemin du fichier ZIP contenant d'autres fichiers .sdc
zip_path = r'C:/Users/user/Downloads/Projet 2024.zip'

# Appeler la fonction pour traiter l'archive ZIP
df_combined = process_zip(zip_path)

# Charger les données dans SQL Server
if not df_combined.empty:
    # Nettoyer le DataFrame
    df_combined = clean_dataframe(df_combined)

    load_to_sql_server(
        df=df_combined,
        table_name='Producti',
        server='DESKTOP-U56S90R',
        database='KomaxData',
        username='sa',
        password='yosr'
    )


Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\01
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\01\Producti.SDC
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\04
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\04\Producti.SDC
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\05
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\05\Producti.SDC
Exploration : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\06
Fichier trouvé : C:\Users\user\AppData\Local\Temp\tmpb9ow_pp3\Data_komax\433\11\06\Producti.SDC
Exploration : C:

In [1]:
#_________________________________insertion de New000KomaxData pour charger vers chargeods________________________________
import pandas as pd
from sqlalchemy import create_engine

# Configuration de la connexion SQL Server
server = 'DESKTOP-U56S90R'
database = 'Data_Komax'
username = 'sa'
password = 'yosr'

# Connexion via SQLAlchemy
engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")

# Étape 1: Lire les colonnes de la table source
query = """
SELECT 
      [Section],
      [DateTimeStamp_1],
      [DateTimeStamp_2],
      [ArticleKey],
      [Job_1],
      [Job_2],
      [UserName],
      [SampleRequestedPieces],
      [SampleStartedCounterDateTimeStamp_1],
      [SampleStartedCounterDateTimeStamp_2],
      [SampleStartedCounterWire_1],
      [SampleStartedCounterWire_2],
      [SampleStartedCounterWire_3],
      [SampleStartedCounterTerminal_1],
      [SampleStartedCounterTerminal_2],
      [SampleStartedCounterTerminal_3],
      [SampleTerminatedCounterDateTimeStamp_1],
      [SampleTerminatedCounterDateTimeStamp_2],
      [SampleTerminatedCounterWire_1],
      [SampleTerminatedCounterWire_2],
      [SampleTerminatedCounterWire_3],
      [SampleTerminatedCounterTerminal_1],
      [SampleTerminatedCounterTerminal_2],
      [SampleTerminatedCounterTerminal_3],
      [LearnStartedCounterDateTimeStamp_1],
      [LearnStartedCounterDateTimeStamp_2],
      [LearnStartedCounterWire_1],
      [LearnStartedCounterWire_2],
      [LearnStartedCounterWire_3],
      [LearnStartedCounterTerminal_1],
      [LearnStartedCounterTerminal_2],
      [LearnStartedCounterTerminal_3],
      [LearnAbortedCounterDateTimeStamp_1],
      [LearnAbortedCounterDateTimeStamp_2],
      [LearnAbortedCounterWire_1],
      [LearnAbortedCounterWire_2],
      [LearnAbortedCounterWire_3],
      [LearnAbortedCounterTerminal_1],
      [LearnAbortedCounterTerminal_2],
      [LearnAbortedCounterTerminal_3],
      [LearnTerminatedCounterDateTimeStamp_1],
      [LearnTerminatedCounterDateTimeStamp_2],
      [LearnTerminatedCounterWire_1],
      [LearnTerminatedCounterWire_2],
      [LearnTerminatedCounterWire_3],
      [LearnTerminatedCounterTerminal_1],
      [LearnTerminatedCounterTerminal_2],
      [LearnTerminatedCounterTerminal_3],
      [ProductionRequestedPieces],
      [ProductionStartedCounterDateTimeStamp_1],
      [ProductionStartedCounterDateTimeStamp_2],
      [ProductionStartedCounterWire_1],
      [ProductionStartedCounterWire_2],
      [ProductionStartedCounterWire_3],
      [ProductionStartedCounterTerminal_1],
      [ProductionStartedCounterTerminal_2],
      [ProductionStartedCounterTerminal_3],
      [TotalGoodPieces],
      [ProductionInterruptedCounterDateTimeStamp_1],
      [ProductionInterruptedCounterDateTimeStamp_2],
      [ProductionInterruptedCounterWire_1],
      [ProductionInterruptedCounterWire_2],
      [ProductionInterruptedCounterWire_3],
      [ProductionInterruptedCounterTerminal_1],
      [ProductionInterruptedCounterTerminal_2],
      [ProductionInterruptedCounterTerminal_3],
      [ProductionRestartedCounterDateTimeStamp_1],
      [ProductionRestartedCounterDateTimeStamp_2],
      [ProductionRestartedCounterWire_1],
      [ProductionRestartedCounterWire_2],
      [ProductionRestartedCounterWire_3],
      [ProductionRestartedCounterTerminal_1],
      [ProductionRestartedCounterTerminal_2],
      [ProductionRestartedCounterTerminal_3],
      [UserRequestedPieces],
      [ProductionTerminatedCounterDateTimeStamp_1],
      [ProductionTerminatedCounterDateTimeStamp_2],
      [ProductionTerminatedCounterWire_1],
      [ProductionTerminatedCounterWire_2],
      [ProductionTerminatedCounterWire_3],
      [ProductionTerminatedCounterTerminal_1],
      [ProductionTerminatedCounterTerminal_2],
      [ProductionTerminatedCounterTerminal_3],
      [SampleAbortedCounterDateTimeStamp_1],
      [SampleAbortedCounterDateTimeStamp_2],
      [SampleAbortedCounterWire_1],
      [SampleAbortedCounterWire_2],
      [SampleAbortedCounterWire_3],
      [SampleAbortedCounterTerminal_1],
      [SampleAbortedCounterTerminal_2],
      [SampleAbortedCounterTerminal_3],
      [SampleStartedCounterSeal_1],
      [SampleStartedCounterSeal_2],
      [SampleStartedCounterSeal_3],
      [SampleTerminatedCounterSeal_1],
      [SampleTerminatedCounterSeal_2],
      [SampleTerminatedCounterSeal_3],
      [QualityParametersCounterDateTimeStamp_1],
      [QualityParametersCounterDateTimeStamp_2],
      [QualityParametersCounterWire_1],
      [QualityParametersCounterWire_2],
      [QualityParametersCounterWire_3],
      [QualityParametersCounterTerminal_1],
      [QualityParametersCounterTerminal_2],
      [QualityParametersCounterTerminal_3],
      [Event],
      [LearnStartedCounterSeal_1],
      [LearnStartedCounterSeal_2],
      [LearnStartedCounterSeal_3],
      [LearnTerminatedCounterSeal_1],
      [LearnTerminatedCounterSeal_2],
      [LearnTerminatedCounterSeal_3],
      [ProductionPieces_1],
      [ProductionPieces_2],
      [ProductionStartedCounterSeal_1],
      [ProductionStartedCounterSeal_2],
      [ProductionStartedCounterSeal_3],
      [ProductionInterruptedCounterSeal_1],
      [ProductionInterruptedCounterSeal_2],
      [ProductionInterruptedCounterSeal_3],
      [Seal],
      [ProductionRestartedCounterSeal_1],
      [ProductionRestartedCounterSeal_2],
      [ProductionRestartedCounterSeal_3]
FROM [dbo].[KomaxData]
"""

# Lire les données
data = pd.read_sql_query(query, engine)

# Étape 2: Insérer les données dans une nouvelle table
#table_cible = 'New000KomaxData'
table_cible = 'Producti'

# Insérer directement en utilisant SQLAlchemy
data.to_sql(name=table_cible, con=engine, if_exists='append', index=False, chunksize=1000)

print("Les données ont été transférées avec succès.")


Les données ont été transférées avec succès.
